In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1) We import all the libraries necessay

We import pandas for reading and handling the data. Numpy provides us with very efficient ways to perform mathematical operations on arrays as well as easy array creation. Sklearn provides us with the tools that we need to build a model in the later stages of the pipeline. Seaborn and matplotlab are two important and highly used libraries for displaying and plotting graphs

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor


#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

In [ ]:
import seaborn as sns

# 2) Reading the data

In [ ]:
name = ['Time','ActivePower', 'AmbientTemperatue', 'BearingShaftTemperature', 'Blade1PitchAngle', 'Blade2PitchAngle', 'Blade3PitchAngle', 'ControlBoxTemperature', 'GearboxBearingTemperature', 'GearboxOilTemperature', 'GeneratorRPM', 'GeneratorWinding1Temperature', 'GeneratorWinding2Temperature', 'HubTemperature', 'MainBoxTemperature', 'NacellePosition', 'ReactivePower', 'RotorRPM', 'TurbineStatus', 'WTG', 'WindDirection']

df = pd.read_csv('../input/wind-power-forecasting/Turbine_Data.csv',names = name, parse_dates = True, header = 0)

df['Time'] = pd.to_datetime(df.index, yearfirst = True,)

In [ ]:
fiddler1 = pd.read_csv('../input/wind-power-forecasting/Turbine_Data.csv', parse_dates = True, index_col = 0,header = 0) # here we are making a new copy of the dataset to mess around with.

fiddler1.head()

In [ ]:
fiddler1.drop(columns = ['WTG','ControlBoxTemperature'], inplace = True) #We drop WTG because every value was the same

fiddler1.index = pd.to_datetime(fiddler1.index) # Here we are converting the index to a pd datetime format


In [ ]:
intrap_df = pd.read_csv('../input/wind-power-forecasting/Turbine_Data.csv', parse_dates = True, index_col = 0,header = 0)

intrap_df.drop(columns = ['WTG','ControlBoxTemperature'], inplace = True) #We drop WTG because every value was the same

intrap_df.index = pd.to_datetime(fiddler1.index) # Here we are converting the index to a pd datetime format



In [ ]:
mode_df = intrap_df.copy()

forward_fill = intrap_df.copy()

# 4) Handling null values

In [ ]:
fiddler1.info()
print('-'*30)

fiddler1.isnull().sum() #we use this to check for missing values

We can see that there are many missing values which would either need to be filled or removed. We can also infer that most of the columns are floats, so they dont need to be encoded anyhow. However WTG column, is of the object type, so we need to handle that as well.

In [ ]:
target = ['ActivePower']

In [ ]:
columns = ['AmbientTemperatue' ,'BearingShaftTemperature', 'Blade1PitchAngle', 'Blade2PitchAngle' ,'Blade3PitchAngle','GearboxBearingTemperature', 'GearboxOilTemperature','GeneratorRPM','GeneratorWinding1Temperature','GeneratorWinding2Temperature','HubTemperature','MainBoxTemperature','NacellePosition','ReactivePower','RotorRPM','TurbineStatus', 'WindDirection','WindSpeed']


for n in columns:
    fiddler1[n].fillna(fiddler1[n].median(),inplace = True)
    intrap_df[n].fillna(intrap_df[n].mean(),inplace = True)
    mode_df[n].fillna(mode_df[n].mode(),inplace = True)
    


In [ ]:
fiddler1.describe(datetime_is_numeric = True) #We can see our data summarized from a statistical perspective with the describe() function

In [ ]:
fiddler1.info()
print('-'*30)

fiddler1.isnull().sum() #we use this to check for missing values


intrap_df.isnull().sum()

mode_df.info()

We can see that after filling all the null values with it's median we were able to get rid of many of the empty columns. Now we have only 2 columns with null values. Every value in the ControlBoxTemperature column is 0 as we can see in the data, therefore we deem it to be redundant and drop it entirely. For the ActivePower we can drop all the rows with null values.

In [ ]:
# code to drop the rows where the Active Power is not available.

df_all = [fiddler1,intrap_df,forward_fill]

for df in df_all:
    df.dropna(axis=0,how = 'any',inplace = True)
    print(df.isnull().sum())



As we can see, we have managed to get rid of all the null values, now let us perform EDA and then build a model. Since all the values are a number we dont need to perform  categorical imputation.

In [ ]:
forward_fill.head()

# 4) Performing EDA

In [ ]:
pd.unique(list(fiddler1.index.year)) # here we are checking the years present in our data.

We will now extract the month and the year from the index, and create two new columns.

In [ ]:
fiddler1['Year'] = fiddler1.index.year
fiddler1['month'] = fiddler1.index.month
fiddler1['hour'] = fiddler1.index.hour

In [ ]:
fiddler1.loc['2018']

# 5) Visualization

Using the heatmap function provided by the seaborn library 

In [ ]:
sns.heatmap(fiddler1.corr()) #Using corr() we can find out how the variables are related to each other

In [ ]:
sns.heatmap(forward_fill.corr())

# 6) Building a model

In [ ]:
column = ['AmbientTemperatue' ,'BearingShaftTemperature', 'Blade1PitchAngle', 'Blade2PitchAngle' ,'Blade3PitchAngle','GearboxBearingTemperature', 'GearboxOilTemperature','GeneratorRPM','GeneratorWinding1Temperature','GeneratorWinding2Temperature','HubTemperature','MainBoxTemperature','NacellePosition','ReactivePower','RotorRPM','TurbineStatus', 'WindDirection']

In [ ]:
x_median = forward_fill[columns]

y_median = forward_fill[target]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV


    

In [ ]:


#remove the hashtag in the line below for train_test_split()
x_train,x_test,y_train,y_test = train_test_split(x_median,y_median,test_size=0.3, random_state=1)

#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )

In [ ]:
x_train

Here we are normalizing our dataset, so that the difference in units doesn't affect our model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler


norm = MinMaxScaler().fit(x_train)

xtrain_norm = norm.transform(x_train)

xtest_norm = norm.transform(x_test)

In [ ]:
xtrain_norm = pd.DataFrame(xtrain_norm)
xtest_norm = pd.DataFrame(xtest_norm)

In [ ]:
from sklearn                        import metrics, svm
from sklearn.linear_model           import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


def LinearRegressionModel(xtrain,xtest,ytrain,ytest):
    
    pca = PCA()

    pca.fit(xtrain)

    pca_train = pca.transform(xtrain)
    pca_test = pca.transform(xtest)


    X_new = SelectKBest(f_regression, k=10).fit_transform(xtrain,ytrain)

    X_new_test = SelectKBest(f_regression, k=10).fit_transform(xtest,ytest) 
    
    clf = LinearRegression()
    clf.fit(pca_train, y_train)
    
    print("LinearRegression")
    pred = clf.predict(pca_test)

    pred_train = clf.predict(pca_train)
    
    print("Test Evaluation: ")
    print(np.sqrt(mean_squared_error(ytest,pred)))
    print(r2_score(y_test,pred))
    print("Train Evaluation: \n")
    print(np.sqrt(mean_squared_error(ytrain,pred_train)))
    print(r2_score(y_train,pred_train))
    
    return 






In [ ]:
x_new = pd.DataFrame(X_new)

In [ ]:
LinearRegressionModel(xtrain_norm,xtest_norm,y_train,y_test)
